In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs 
from splinter import Browser
import requests
import time
from webdriver_manager.chrome import ChromeDriverManager
import pymongo

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======


[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/88.0.4324.96/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\Stephen\.wdm\drivers\chromedriver\win32\88.0.4324.96]


In [3]:
#Visits nasa news webpage for mars
url_news = "https://mars.nasa.gov/news/"

browser.visit(url_news)

In [6]:
#Scrape news site for News Title and Paragraph Text
html_news = browser.html
soup = bs(html_news, "html.parser")
#article allows new 
article = soup.find("div", class_='list_text')


#Titles contained in <div class="content_title"
title = article.find("div", class_ = "content_title").text


#Pragraph text contained in <div class="article_teaser_body"
paragraph = article.find("div", class_ = "article_teaser_body").text

In [7]:
# Display scraped info
print(title)
print(paragraph)

NASA's Mars Helicopter Reports In 
The technology demonstration has phoned home from where it is attached to the belly of NASA’s Perseverance rover. 


In [8]:
#JPL space images
url_spaceimage = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url_spaceimage)


In [9]:
img_html = browser.html
img_soup = bs(img_html, "html.parser")

In [13]:
# Find image url to the full size
featured_image = img_soup.find("img", class_="BaseImage object-contain")["data-src"]
    
# Display url of the full image
print(featured_image)

https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA23726.2e16d0ba.fill-400x400-c50.jpg


In [20]:
#Visits the Mars Facts webpage
url_mars_facts = "https://space-facts.com/mars/"
browser.visit(url_mars_facts)

mars_facts = pd.read_html(url_mars_facts)
mars_facts

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [19]:
mars_facts_df = mars_facts[0]

mars_facts_df.columns = ["Fact", "Value"]

#sets index to fact column 
mars_facts_df.set_index("Fact", inplace=True)

mars_facts_df

,Value
Fact,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [21]:
#save table to html
mars_facts_df.to_html("mars_facts_data.html")

In [24]:
#Mars Hemispheres
hemispheres_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemispheres_url)

#HTML object
html_hemisphere = browser.html
soup = bs(html_hemisphere, "html.parser")

In [28]:
# Scrape all items that contain mars hemispheres information
hemispheres = soup.find_all("div", class_="item")

# empty dictionary to be appended
mars_hemispheres = []
hemispheres_url = "https://astrogeology.usgs.gov"

# Loop through the list of all hemispheres information
for hemisphere in hemispheres:
    title = hemisphere.find("h3").text
    hemispheres_img = hemisphere.find("a", class_="itemLink product-item")["href"]
    
    # Visit the link that contains the full image website 
    browser.visit(hemispheres_url + hemispheres_img)
    
    # HTML Object
    image_html = browser.html
    image_info = bs(image_html, "html.parser")
    
    # Create full image url
    hemisphere_url = hemispheres_url + image_info.find("img", class_="wide-image")["src"]
    
    mars_hemispheres.append({"title" : title, "img_url" : hemisphere_url})

#print title + url
    print(title)
    print(hemisphere_url)
    

Cerberus Hemisphere Enhanced
https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg
Schiaparelli Hemisphere Enhanced
https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg
Syrtis Major Hemisphere Enhanced
https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg
Valles Marineris Hemisphere Enhanced
https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg


In [29]:
mars_hemispheres

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]